In [63]:
from qiskit_aer import AerSimulator
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.drivers import PySCFDriver

import numpy as np
from scipy.sparse.linalg import eigsh
from openfermionpyscf import run_pyscf
from openfermion import MolecularData
from openfermion.transforms import get_fermion_operator, jordan_wigner
from openfermion.linalg import get_sparse_operator

In [64]:
#backend = AerSimulator(method = "statevector",noise_model=None)

In [65]:

basis = 'sto-3g'
H2_atoms = ["H","H","H","H"]
H2_coords = [(0,0,0), (0,0,0.735),(1,0,0), (1,0,0.735)]
H2_charge = 0
H2_multiplicity = 1

H2_moleculeinfo = MoleculeInfo(H2_atoms, H2_coords, charge=H2_charge, multiplicity=H2_multiplicity)

energy_arr =[]
order=[]
driver = PySCFDriver.from_molecule(H2_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
fermionic_hamiltonian = E_problem.hamiltonian
repulsion = fermionic_hamiltonian.constants['nuclear_repulsion_energy']
second_q_op = fermionic_hamiltonian.second_q_op()

In [66]:
def fermion_to_qubit(problem, second_q_op, mapper_name):
    if mapper_name == "JW":
        mapper = JordanWignerMapper()
    if mapper_name == "Pa":
        mapper = ParityMapper(num_particles=problem.num_particles)
    if mapper_name == "BK":
        mapper = BravyiKitaevMapper()
    qubit_op = mapper.map(second_q_op)
    return qubit_op , mapper

In [67]:
print(second_q_op)

Fermionic Operator
number spin orbitals=8, number terms=264
  0.2836719366718984 * ( +_0 +_0 -_0 -_0 )
+ 0.27649113231363276 * ( +_0 +_1 -_1 -_0 )
+ 0.2726646946168434 * ( +_0 +_2 -_2 -_0 )
+ 0.2780314784272836 * ( +_0 +_3 -_3 -_0 )
+ 0.2836719366718984 * ( +_0 +_4 -_4 -_0 )
+ 0.27649113231363276 * ( +_0 +_5 -_5 -_0 )
+ 0.2726646946168434 * ( +_0 +_6 -_6 -_0 )
+ 0.2780314784272836 * ( +_0 +_7 -_7 -_0 )
+ 0.07776459449991521 * ( +_0 +_0 -_1 -_1 )
+ 0.07776459449991521 * ( +_0 +_1 -_0 -_1 )
+ -0.07447301791801932 * ( +_0 +_2 -_3 -_1 )
+ -0.07447301791801932 * ( +_0 +_3 -_2 -_1 )
+ 0.07776459449991521 * ( +_0 +_4 -_5 -_1 )
+ 0.07776459449991521 * ( +_0 +_5 -_4 -_1 )
+ -0.07447301791801932 * ( +_0 +_6 -_7 -_1 )
+ -0.07447301791801932 * ( +_0 +_7 -_6 -_1 )
+ 0.06192837031237733 * ( +_0 +_0 -_2 -_2 )
+ -0.0600859839437311 * ( +_0 +_1 -_3 -_2 )
+ 0.06192837031237733 * ( +_0 +_2 -_0 -_2 )
+ -0.0600859839437311 * ( +_0 +_3 -_1 -_2 )
+ 0.06192837031237733 * ( +_0 +_4 -_6 -_2 )
+ -0.0600859839437

In [68]:
def str_to_bitstate(state_str):
    """
    문자열 ket 상태 "0101..." → int로 변환.
    문자열 왼쪽이 큐빗 N-1, 오른쪽이 큐빗0이라고 가정.
    """
    return int(state_str, 2)

def apply_pauli_string_bits(state, pauli_string, n_qubits):
    """
    state: int로 표현된 ket. ex) 0b0101 → 5
    pauli_string: 문자열. 예) "XIZY..."
    n_qubits: 큐빗 수
    
    Returns:
        (new_state: int, phase: complex)
    """
    phase = 1.0
    # 큐빗0을 오른쪽(Little Endian)으로 가정하므로, pauli_string은 오른쪽부터 큐빗0 → 왼쪽 큐빗N-1
    for i, p in enumerate(reversed(pauli_string)):  # LSB부터 순회
        bit = (state >> i) & 1
        if p == 'I':
            continue
        elif p == 'Z':
            if bit:
                phase *= -1
        elif p == 'X':
            state ^= (1 << i)  # i번째 비트 flip
        elif p == 'Y':
            if bit:
                phase *= -1j
            else:
                phase *= 1j
            state ^= (1 << i)
    return state, phase

def inner_product_pauli_hamiltonian(pauli_op, basis_bra_str, basis_ket_str):
    """
    Compute <Phi_I | H | Phi_J> using Pauli-mapped Hamiltonian.
    pauli_op: Qiskit's PauliSumOp 또는 pauli_op.to_list()를 지원하는 형태
    basis_bra_str, basis_ket_str: "0101..." 형태의 ket 문자열
    
    Returns: float (real part of <bra|H|ket>)
    """
    n_qubits = len(basis_bra_str)
    basis_bra_int = str_to_bitstate(basis_bra_str)
    basis_ket_int = str_to_bitstate(basis_ket_str)
    
    total = 0.0
    for pauli_string, coeff in pauli_op.to_list():
        new_state_int, phase = apply_pauli_string_bits(basis_ket_int, pauli_string, n_qubits)
        if new_state_int == basis_bra_int:
            total += coeff * phase
            
    return total.real

def project_hamiltonian(basis,hamiltonian):
    n = len(basis)
    H = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            basis_bra = basis[i]
            basis_ket = basis[j]
            inner_product = inner_product_pauli_hamiltonian(hamiltonian, basis_bra , basis_ket)
            H_ij = np.real(inner_product)
            H[i, j] = H_ij 

            #print("<{}|H|{}> = {}".format(basis_bra,basis_ket,H_ij))
    return H 

In [69]:
from openfermionpyscf import run_pyscf
from openfermion import MolecularData, get_fermion_operator
from openfermion.linalg import get_sparse_operator


def indices_with_N_and_Sz0(n_qubits, n_elec):
    """전자수=N 이면서 S_z=0 (interleaved: 짝수=alpha, 홀수=beta)"""
    idx = []
    for s in range(1 << n_qubits):
        if s.bit_count() != n_elec:
            continue
        n_alpha = sum((s >> i) & 1 for i in range(0, n_qubits, 2))  # 0,2,4,...
        n_beta  = sum((s >> i) & 1 for i in range(1, n_qubits, 2))  # 1,3,5,...
        if n_alpha == n_beta:
            idx.append(s)
    return np.array(idx, dtype=int)


# 1. H2 분자 정의
geometry = [('H', (0., 0., 0.)),
            ('H', (0., 0., 0.7414))]  # Angstrom
basis = 'sto-3g'
mol = MolecularData(geometry, basis, multiplicity=1, charge=0)

# 2. PySCF 실행 (SCF까지만 해도 충분)
mol = run_pyscf(mol, run_scf=1, run_fci=1)

# 3. 2차 정량화 Hamiltonian 얻기
ham_int = mol.get_molecular_hamiltonian()
ham_fci = get_fermion_operator(ham_int)

print("Number of terms:", len(ham_fci.terms))
for term, coeff in list(ham_fci.terms.items())[:10]:
    print(term, coeff)
    
    
H = get_sparse_operator(ham_fci, n_qubits=mol.n_qubits)
print(H)

evals, evecs = eigsh(H, k=1, which='SA')   # Hermitian이므로 OK
print(evals)


Number of terms: 37
() 0.7137539936876182
((0, 1), (0, 0)) -1.2524635735648981
((1, 1), (1, 0)) -1.2524635735648981
((2, 1), (2, 0)) -0.4759487152209644
((3, 1), (3, 0)) -0.4759487152209644
((0, 1), (0, 1), (0, 0), (0, 0)) 0.33724438317841876
((0, 1), (0, 1), (2, 0), (2, 0)) 0.09064440410574796
((0, 1), (1, 1), (1, 0), (0, 0)) 0.33724438317841876
((0, 1), (1, 1), (3, 0), (2, 0)) 0.09064440410574796
((0, 1), (2, 1), (0, 0), (2, 0)) 0.09064440410574796
<Compressed Sparse Column sparse matrix of dtype 'complex128'
	with 20 stored elements and shape (16, 16)>
  Coords	Values
  (0, 0)	(0.7137539936876182+0j)
  (1, 1)	(0.23780527846665378+0j)
  (2, 2)	(0.23780527846665378+0j)
  (3, 3)	(0.4592503306687161+0j)
  (12, 3)	(0.18128880821149593+0j)
  (4, 4)	(-0.53870957987728+0j)
  (5, 5)	(-0.5324790068861723+0j)
  (6, 6)	(-0.35119019867467643+0j)
  (9, 6)	(-0.18128880821149593+0j)
  (7, 7)	(0.3524341417394577+0j)
  (8, 8)	(-0.53870957987728+0j)
  (6, 9)	(-0.18128880821149593+0j)
  (9, 9)	(-0.3511

In [71]:
# 2) N=2, S_z=0 서브스페이스 (H2/STO-3G면 4x4)
idx_N2_Sz0 = indices_with_N_and_Sz0(mol.n_qubits, n_elec=4)
print(idx_N2_Sz0)
H_N2_Sz0 = H[idx_N2_Sz0, :][:, idx_N2_Sz0].tocsr()
print(H_N2_Sz0)
print("N=2, Sz=0 block shape:", H_N2_Sz0.shape)  # (4, 4)

[15]
<Compressed Sparse Row sparse matrix of dtype 'complex128'
	with 1 stored elements and shape (1, 1)>
  Coords	Values
  (0, 0)	(0.9201067191670369+0j)
N=2, Sz=0 block shape: (1, 1)
